In [6]:
import duckdb
import os
import pandas as pd
from datetime import datetime
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import pyarrow as pa
import pyarrow.parquet as pq
import glob

In [13]:
# Chemin vers les fichiers
input_folder = "../data/cleaned/"
output_path = "../data/output/data-sampled.parquet"

# Lire tous les fichiers .parquet dans le dossier
file_paths = glob.glob(f"{input_folder}/*.parquet")

# Initialiser une base de données en mémoire
db = duckdb.connect(database=':memory:')

# Initialisation de la table finale dynamique (en fonction des colonnes du premier fichier)
first_file = file_paths[0]
db.execute(f"CREATE TABLE final_sampled AS SELECT * FROM parquet_scan('{first_file}') LIMIT 0")

# Liste pour accumuler les données par morceaux
batch_writer = None

for file_path in file_paths:
    print(f"\nTraitement du fichier : {file_path}")

    # Supprimer la table temporaire si elle existe déjà
    db.execute("DROP TABLE IF EXISTS temp_sampled")

    # Échantillonnage direct sans charger tout le fichier en mémoire
    db.execute(f"""
        CREATE TEMP TABLE temp_sampled AS
        SELECT * FROM parquet_scan('{file_path}') USING SAMPLE 3%
    """)
    print("- Échantillonnage de 3% des données dans temp_sampled")

    # Extraire et stocker user_id dans un DataFrame
    user_ids = db.execute("SELECT DISTINCT user_id FROM temp_sampled WHERE user_id IS NOT NULL").fetch_df()['user_id'].values

    if len(user_ids) > 0:
        np.random.shuffle(user_ids)

        # Extraire les lignes où user_id est NULL
        null_user_ids = db.execute("SELECT rowid FROM temp_sampled WHERE user_id IS NULL").fetch_df()['rowid'].values

        # Assigner un user_id aléatoire unique à chaque ligne où user_id est NULL
        for rowid in null_user_ids:
            random_user_id = np.random.choice(user_ids)
            db.execute(f"UPDATE temp_sampled SET user_id = '{random_user_id}' WHERE rowid = {rowid}")

        print("- Attribution aléatoire des user_id dans temp_sampled")
    else:
        print("- Avertissement : Aucun user_id valide trouvé dans temp_sampled")

    # Insérer les données échantillonnées dans la table finale par lots
    db.execute("INSERT INTO final_sampled SELECT * FROM temp_sampled")
    print("- Données insérées dans final_sampled")

# Exporter le résultat final sans tout charger en mémoire
# Écrire les résultats par petits morceaux
chunk_size = 100000  # Taille des morceaux à exporter
offset = 0

while True:
    result = db.execute(f"SELECT * FROM final_sampled WHERE user_id IS NOT NULL LIMIT {chunk_size} OFFSET {offset}").fetch_df()

    if result.empty:
        break

    # Convertir le DataFrame Pandas en table Arrow pour l'écriture Parquet
    table = pa.Table.from_pandas(result)

    # Si le fichier n'existe pas, créer un nouveau fichier Parquet
    if batch_writer is None:
        batch_writer = pq.ParquetWriter(output_path, table.schema)

    # Ajouter le lot au fichier Parquet
    batch_writer.write_table(table)
    print(f"- Exporté {len(result)} lignes à partir de l'offset {offset}")

    offset += chunk_size

# Fermer l'écrivain Parquet pour finaliser l'écriture
if batch_writer:
    batch_writer.close()

print(f"\nFichier sauvegardé : {output_path}")


Traitement du fichier : ../data/cleaned\2019-Dec.parquet
- Échantillonnage de 3% des données dans temp_sampled
- Attribution aléatoire des user_id dans temp_sampled
- Données insérées dans final_sampled

Traitement du fichier : ../data/cleaned\2019-Nov.parquet
- Échantillonnage de 3% des données dans temp_sampled
- Attribution aléatoire des user_id dans temp_sampled
- Données insérées dans final_sampled

Traitement du fichier : ../data/cleaned\2019-Oct.parquet
- Échantillonnage de 3% des données dans temp_sampled
- Attribution aléatoire des user_id dans temp_sampled
- Données insérées dans final_sampled

Traitement du fichier : ../data/cleaned\2020-Apr.parquet
- Échantillonnage de 3% des données dans temp_sampled
- Attribution aléatoire des user_id dans temp_sampled
- Données insérées dans final_sampled

Traitement du fichier : ../data/cleaned\2020-Feb.parquet
- Échantillonnage de 3% des données dans temp_sampled
- Attribution aléatoire des user_id dans temp_sampled
- Données insérées

In [8]:
# 📂 Définition des chemins
DATA_PATH = "../data/output/data-sampled"
OUTPUT_PATH = "../data/output/user_features_data-sampled.parquet"

In [9]:
# 📌 Catégories sélectionnées
SELECTED_CATEGORIES = {"electronics", "computers", "sport", "kids"}

def load_data_with_duckdb():
    """Charge et filtre les données en streaming avec DuckDB pour économiser la mémoire."""
    print("📂 Chargement des fichiers Parquet en streaming avec DuckDB...")

    query = f"""
        SELECT 
            user_id, 
            event_type, 
            event_time,
            price, 
            split_part(category_code, '.', 1) AS category_main
        FROM read_parquet('{DATA_PATH}*.parquet', hive_partitioning=0)
        WHERE event_type IN ('view', 'purchase','cart')
        AND split_part(category_code, '.', 1) IN ({', '.join(f"'{cat}'" for cat in SELECTED_CATEGORIES)})
    """

    return duckdb.query(query)  # Retourne un objet DuckDB en streaming

def generate_user_features():
    """Transforme les données en jeu de caractéristiques par utilisateur avec DuckDB en optimisant la mémoire."""
    print("🔄 Chargement des données en streaming...")
    df = load_data_with_duckdb()  # Chargement optimisé

    print("📊 Calcul des métriques par utilisateur...")

    # 📌 Agrégation en plusieurs étapes pour réduire la mémoire utilisée
    user_features_query = """
        SELECT 
            user_id,
            COUNT(*) AS total_events,
            COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS total_views,
            COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS total_carts,
            COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS total_purchases,
            SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS total_spent,
            COUNT(DISTINCT category_main) AS unique_categories,
            MAX(event_time) AS last_event_time
        FROM df
        GROUP BY user_id
    """

    user_features = duckdb.query(user_features_query).to_df()

    print("📅 Conversion des dates...")
    # 📌 Conversion des dates (en UTC et naive)
    user_features["last_event_time"] = pd.to_datetime(user_features["last_event_time"]).dt.tz_localize(None)

    print("🔢 Calcul du taux de conversion...")
    # 📌 Calcul du taux de conversion
    user_features["conversion_rate_view"] = user_features["total_purchases"] / user_features["total_views"]
    user_features["conversion_rate_view"].fillna(0, inplace=True)

    print("🔢 Calcul du taux de panier transformé...")
    # 📌 Calcul du taux de conversion
    user_features["conversion_rate_cart"] = user_features["total_purchases"] / user_features["total_carts"]
    user_features["conversion_rate_cart"].fillna(0, inplace=True)

    print("⏳ Calcul du temps depuis le dernier événement...")
    # 📌 Calcul du temps depuis le dernier événement (en jours)
    now_utc = datetime.utcnow()
    user_features["days_since_last_event"] = (
        now_utc - user_features["last_event_time"]
    ).dt.total_seconds() / (3600 * 24)

    # 📌 Nettoyage des valeurs infinies et NaN
    user_features.replace([float("inf"), float("-inf")], None, inplace=True)
    user_features.fillna(0, inplace=True)

    # 📌 Sauvegarde avec compression (pour économiser la RAM)
    print(f"💾 Sauvegarde optimisée du fichier {OUTPUT_PATH}...")
    os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
    user_features.to_parquet(OUTPUT_PATH, engine="pyarrow", compression="snappy")

    print("✅ Transformation terminée avec succès !")

if __name__ == "__main__":
    generate_user_features()

🔄 Chargement des données en streaming...
📂 Chargement des fichiers Parquet en streaming avec DuckDB...
📊 Calcul des métriques par utilisateur...
📅 Conversion des dates...
🔢 Calcul du taux de conversion...
🔢 Calcul du taux de panier transformé...
⏳ Calcul du temps depuis le dernier événement...
💾 Sauvegarde optimisée du fichier ../data/output/user_features_data-sampled.parquet...


C:\Users\Pierre\AppData\Local\Temp\ipykernel_2872\2321329801.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_features["conversion_rate_view"].fillna(0, inplace=True)
C:\Users\Pierre\AppData\Local\Temp\ipykernel_2872\2321329801.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

✅ Transformation terminée avec succès !


In [11]:
# 🔹 Charger les modèles sauvegardés
print("📥 Chargement des modèles sauvegardés...")
scaler = joblib.load("../data/output/scaler-test.pkl")
pca = joblib.load("../data/output/pca_model-test.pkl")
kmeans = joblib.load("../data/output/kmeans_model-test.pkl")
print("✅ Modèles chargés : scaler, PCA, et K-Means.")

# 🔹 Charger les nouvelles données
print("📥 Chargement des nouvelles données...")
new_data = pd.read_parquet("../data/output/user_features_data-sampled.parquet")
print(f"✅ Nouvelles données chargées : {new_data.shape[0]} lignes, {new_data.shape[1]} colonnes.")

# 🔹 Normaliser les nouvelles données
print("🔄 Normalisation des nouvelles données...")
features = ["total_events", "total_views", "total_carts", "total_purchases", 
            "total_spent", "unique_categories", "days_since_last_event"]
new_data_scaled = scaler.transform(new_data[features])  # Appliquer le scaler sauvegardé
print("✅ Normalisation terminée.")

# 🔹 Réduction de dimension avec l'ACP
print("🔄 Réduction de dimension avec l'ACP...")
new_data_pca = pca.transform(new_data_scaled)  # Appliquer le modèle PCA sauvegardé
new_data_pca_df = pd.DataFrame(data=new_data_pca, columns=["PC1", "PC2"])
new_data_pca_df["user_id"] = new_data["user_id"]
print("✅ Réduction de dimension terminée.")

# 🔹 Prédire les clusters
print("🎯 Prédiction des clusters...")
new_data_pca_df["Cluster"] = kmeans.predict(new_data_pca_df[["PC1", "PC2"]])  # Appliquer le modèle K-Means
print("✅ Prédiction des clusters terminée.")

# 🔹 Sauvegarder les résultats
print("💾 Sauvegarde des résultats...")
new_data_pca_df.to_parquet("../data/output/new_user_clusters-test.parquet", index=False)
print("✅ Résultats sauvegardés dans '../data/output/new_user_clusters-test.parquet'.")

📥 Chargement des modèles sauvegardés...
✅ Modèles chargés : scaler, PCA, et K-Means.
📥 Chargement des nouvelles données...
✅ Nouvelles données chargées : 814509 lignes, 11 colonnes.
🔄 Normalisation des nouvelles données...
✅ Normalisation terminée.
🔄 Réduction de dimension avec l'ACP...
✅ Réduction de dimension terminée.
🎯 Prédiction des clusters...
✅ Prédiction des clusters terminée.
💾 Sauvegarde des résultats...
✅ Résultats sauvegardés dans '../data/output/new_user_clusters-test.parquet'.


In [12]:
import duckdb
import pandas as pd

# Chemin vers votre fichier Parquet
file_path = "../data/output/new_user_clusters-test.parquet"

try:
    # Établir une connexion à DuckDB
    con = duckdb.connect(database=':memory:', read_only=False)

    # Exécuter la requête SQL pour faire le group by et compter les lignes
    result = con.execute(f"SELECT Cluster, COUNT(*) AS count FROM '{file_path}' GROUP BY Cluster").fetchdf()

    # Afficher le résultat
    print(result)

    # Fermer la connexion DuckDB
    con.close()

except duckdb.Error as e:
    print(f"Une erreur DuckDB s'est produite : {e}")
except FileNotFoundError:
    print(f"Le fichier '{file_path}' n'a pas été trouvé.")
except Exception as e:
    print(f"Une erreur inattendue s'est produite : {e}")

   Cluster   count
0        0   49107
1        1     707
2        2  764218
3        3     477
